# Building Data Genome 2.0
## All-meters datasets creation: hourly and daily timestamp

Biam! (pic.biam@gmail.com)

In [1]:
import pandas as pd
import numpy as np
from glob import glob

# Meters data

To join all meters raw dataets run the following code.

## Data join

In [29]:
# Write custom path here
# ken: change \\ to /
# ken: change processed to cleaned
path_raw = "../data/meters/raw/"
path_proc = "../data/meters/cleaned/"

# files in directory
files = glob(path_raw + "*.csv")

files

['../data/meters/raw/hotwater.csv',
 '../data/meters/raw/gas.csv',
 '../data/meters/raw/solar.csv',
 '../data/meters/raw/electricity.csv',
 '../data/meters/raw/water.csv',
 '../data/meters/raw/chilledwater.csv',
 '../data/meters/raw/irrigation.csv',
 '../data/meters/raw/steam.csv']

In [22]:
dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("/")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
complete_data = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

In [24]:
complete_data.head()

,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Robin_public_Carolina,0.2421,hotwater
1,2016-01-01 01:00:00,Robin_public_Carolina,9.7579,hotwater
2,2016-01-01 02:00:00,Robin_public_Carolina,0.0000,hotwater
3,2016-01-01 03:00:00,Robin_public_Carolina,0.0000,hotwater
4,2016-01-01 04:00:00,Robin_public_Carolina,10.0000,hotwater


In [25]:
# Check the meters
complete_data["meter"].value_counts()

electricity     27684432
chilledwater     9736920
steam            6491280
hotwater         3245640
gas              3105288
water            2561424
irrigation        649128
solar              87720
Name: meter, dtype: int64

In [26]:
# Check number of buildings
len(np.unique(complete_data["building_id"]))

1636

In [27]:
# Rounds to 4 decimals
complete_data["meter_reading"] = round(complete_data["meter_reading"],4)

## Hourly timestamp

Meters reading are currently hourly.

In [30]:
complete_data.to_csv(path_proc + "allmeters.csv", index=False)

## Daily timestamp

Daily readings are summarized as the sum of all the readings of that day.

In [ ]:
# Converts timestamp to datetime object
complete_data["timestamp"] = pd.to_datetime(complete_data["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [ ]:
complete_data.info()

In [ ]:
# Sums all readings per day
complete_data = complete_data.groupby(by=["building_id","meter",complete_data['timestamp'].dt.date]).sum()

In [ ]:
# Reset indexes to ungroup
complete_data = complete_data.reset_index()

In [ ]:
complete_data.head()

In [ ]:
# export csv
complete_data.to_csv(path_proc + "allmeters_daily.csv", index=False)

In [ ]:
del complete_data